In [ ]:
import pandas as pd
import numpy as np

# Load the raw data
df = pd.read_csv('/content/raw data (1).csv')  # Replace with your file path

# Remove the first row
df = df.iloc[0:3000]  # Method 1

df.index = range(1, len(df) + 1)


In [ ]:
df

,Incremental_ID,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1,1,E0,8/13/2005,NaN,Aston Villa,Bolton,2,2,D,2,...,2,6,14,16,7,8,0,2,0,0
2,2,E0,8/13/2005,NaN,Everton,Man United,0,2,A,0,...,5,5,15,14,8,6,3,1,0,0
3,3,E0,8/13/2005,NaN,Fulham,Birmingham,0,0,D,0,...,7,4,12,13,6,6,1,2,0,0
4,4,E0,8/13/2005,NaN,Man City,West Brom,0,0,D,0,...,8,3,13,11,3,6,2,3,0,0
5,5,E0,8/13/2005,NaN,Middlesbrough,Liverpool,0,0,D,0,...,2,7,17,11,5,0,2,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,336,E0,4/21/2013,NaN,Tottenham,Man City,3,1,H,0,...,8,7,11,10,4,5,3,3,0,0
2997,337,E0,4/22/2013,NaN,Man United,Aston Villa,3,0,H,3,...,5,5,10,6,5,3,1,0,0,0
2998,338,E0,4/27/2013,NaN,Everton,Fulham,1,0,H,1,...,10,2,12,16,7,2,1,3,0,0
2999,339,E0,4/27/2013,NaN,Man City,West Ham,2,1,H,1,...,10,6,9,7,9,4,1,3,0,0


In [ ]:
def print_last_n_matches(df, team_name, current_index, n=5):
    # Filter matches before the current index where team played (home or away)
    past_matches = df.iloc[:current_index-1][(df['HomeTeam'] == team_name) | (df['AwayTeam'] == team_name)]

    # Get the last N matches
    last_n = past_matches.tail(n)

    # Display
    print(f"Last {n} matches of {team_name} before match index {current_index}:\n")
    print(last_n[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']])  # Add more columns if needed


In [ ]:
print_last_n_matches(df, 'Arsenal', 103, n=5)


Last 5 matches of Arsenal before match index 103:

         Date   HomeTeam    AwayTeam  FTHG  FTAG FTR
56 2005-09-19    Arsenal     Everton     2     0   H
64 2005-09-24   West Ham     Arsenal     0     0   D
72 2005-10-02    Arsenal  Birmingham     1     0   H
82 2005-10-15  West Brom     Arsenal     2     1   H
87 2005-10-22    Arsenal    Man City     1     0   H


<ipython-input-22-6a41f58eb8d7>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  past_matches = df.iloc[:current_index-1][(df['HomeTeam'] == team_name) | (df['AwayTeam'] == team_name)]


In [ ]:
def get_last_n_goals(df, team_name, current_index, n=5):
    # Filter matches before current_index where the team played
    past_matches = df.iloc[:current_index-1][(df['HomeTeam'] == team_name) | (df['AwayTeam'] == team_name)]

    # Get the last n matches
    last_n = past_matches.tail(n)

    # Initialize goal count
    goals = 0

    # Iterate through the last n matches
    for _, match in last_n.iterrows():
        if match['HomeTeam'] == team_name:
            goals += int(match['FTHG'])  # Goals scored at home
        else:
            goals += int(match['FTAG'])  # Goals scored away

    return goals

In [ ]:
goals_last_5 = get_last_n_goals(df, 'Arsenal', 103, n=5)
print(f"Goals scored by Arsenal in their last 5 matches before match 100: {goals_last_5}")


Goals scored by Arsenal in their last 5 matches before match 100: 5


<ipython-input-24-6edeabb1f209>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  past_matches = df.iloc[:current_index-1][(df['HomeTeam'] == team_name) | (df['AwayTeam'] == team_name)]


In [ ]:
def print_team_goals_last_n(df, match_index, n=5):
    match = df.iloc[match_index-1]
    home_team = match['HomeTeam']
    away_team = match['AwayTeam']

    # Filter past matches for each team
    past_home = df.iloc[:match_index-1][(df['HomeTeam'] == home_team) | (df['AwayTeam'] == home_team)].tail(n)
    past_away = df.iloc[:match_index-1][(df['HomeTeam'] == away_team) | (df['AwayTeam'] == away_team)].tail(n)

    # Calculate goals scored
    home_goals = past_home.apply(lambda row: row['FTHG'] if row['HomeTeam'] == home_team else row['FTAG'], axis=1).sum()
    away_goals = past_away.apply(lambda row: row['FTHG'] if row['HomeTeam'] == away_team else row['FTAG'], axis=1).sum()

    print(f"Match {match_index}: {home_team} vs {away_team}")
    print(f"{home_team} goals in last {n} matches: {home_goals}")
    print(f"{away_team} goals in last {n} matches: {away_goals}")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class Arpit:
    def __init__(self, df, match_index, n=5):
        self.df = df
        self.match_index = match_index
        self.n = n
        self.stats = {}
        self.run_functions()


    def make_stat_name(self, col, side):
        return f"{col}_L{self.n}_{side}"

    def get_match_teams(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()
        return home_team, away_team, past_matches

    def compute_stat(self, matches, team, home_col, away_col):
        return matches.apply(lambda row: row[home_col] if row['HomeTeam'] == team else row[away_col], axis=1).sum()


    def run_functions(self):
        self.print_team_goals_last_n()
        self.print_team_shots_last_n()
        self.print_team_shots_target_last_n()
        self.print_team_fouls_last_n()
        self.print_team_corners_last_n()
        self.print_team_yellow_cards_last_n()
        self.print_team_red_cards_last_n()
        self.print_team_win_last_n()
        self.print_team_lose_last_n()
        self.print_team_draw_last_n()
    def get_match_teams(self):
        match = self.df.iloc[self.match_index - 1]
        return match['HomeTeam'], match['AwayTeam'], self.df.iloc[:self.match_index - 1].copy()

    def get_stats(self):
        return self.stats

    def compute_stat(self, past_matches, team, home_col, away_col):
        return past_matches.apply(
            lambda row: row[home_col] if row['HomeTeam'] == team else row[away_col], axis=1
        ).sum()

    def print_team_goals_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_goals = past_home.apply(
            lambda row: row['FTHG'] if row['HomeTeam'] == home_team else row['FTAG'], axis=1
        ).sum()

        away_goals = past_away.apply(
            lambda row: row['FTHG'] if row['HomeTeam'] == away_team else row['FTAG'], axis=1
        ).sum()

        print(f"Match {self.match_index}: {home_team} vs {away_team}")
        print(f"{home_team} goals in last {self.n} matches: {home_goals}")
        print(f"{away_team} goals in last {self.n} matches: {away_goals}")

    def print_team_shots_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_shots = past_home.apply(
            lambda row: row['HS'] if row['HomeTeam'] == home_team else row['AS'], axis=1
        ).sum()

        away_shots = past_away.apply(
            lambda row: row['HS'] if row['HomeTeam'] == away_team else row['AS'], axis=1
        ).sum()

        print(f"{home_team} shots in last {self.n} matches: {home_shots}")
        print(f"{away_team} shots in last {self.n} matches: {away_shots}")

    def print_team_shots_target_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_shots_target = past_home.apply(
            lambda row: row['HST'] if row['HomeTeam'] == home_team else row['AST'], axis=1
        ).sum()

        away_shots_target = past_away.apply(
            lambda row: row['HST'] if row['HomeTeam'] == away_team else row['AST'], axis=1
        ).sum()

        print(f"{home_team} shots on target in last {self.n} matches: {home_shots_target}")
        print(f"{away_team} shots on target in last {self.n} matches: {away_shots_target}")

    def print_team_fouls_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_fouls = past_home.apply(
            lambda row: row['HF'] if row['HomeTeam'] == home_team else row['AF'], axis=1
        ).sum()

        away_fouls = past_away.apply(
            lambda row: row['HF'] if row['HomeTeam'] == away_team else row['AF'], axis=1
        ).sum()

        print(f"{home_team} fouls in last {self.n} matches: {home_fouls}")
        print(f"{away_team} fouls in last {self.n} matches: {away_fouls}")

    def print_team_corners_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_corners = past_home.apply(
            lambda row: row['HC'] if row['HomeTeam'] == home_team else row['AC'], axis=1
        ).sum()

        away_corners = past_away.apply(
            lambda row: row['HC'] if row['HomeTeam'] == away_team else row['AC'], axis=1
        ).sum()

        print(f"{home_team} corners in last {self.n} matches: {home_corners}")
        print(f"{away_team} corners in last {self.n} matches: {away_corners}")

    def print_team_yellow_cards_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_yellows = past_home.apply(
            lambda row: row['HY'] if row['HomeTeam'] == home_team else row['AY'], axis=1
        ).sum()

        away_yellows = past_away.apply(
            lambda row: row['HY'] if row['HomeTeam'] == away_team else row['AY'], axis=1
        ).sum()

        print(f"{home_team} yellow cards in last {self.n} matches: {home_yellows}")
        print(f"{away_team} yellow cards in last {self.n} matches: {away_yellows}")

    def print_team_red_cards_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_reds = past_home.apply(
            lambda row: row['HR'] if row['HomeTeam'] == home_team else row['AR'], axis=1
        ).sum()

        away_reds = past_away.apply(
            lambda row: row['HR'] if row['HomeTeam'] == away_team else row['AR'], axis=1
        ).sum()

        print(f"{home_team} red cards in last {self.n} matches: {home_reds}")
        print(f"{away_team} red cards in last {self.n} matches: {away_reds}")

    def print_team_win_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_wins = 0
        for _, row in past_home.iterrows():
            if row['HomeTeam'] == home_team and row['FTR'] == 'H':
                home_wins += 1
            elif row['AwayTeam'] == home_team and row['FTR'] == 'A':
                home_wins += 1

        away_wins = 0
        for _, row in past_away.iterrows():
            if row['HomeTeam'] == away_team and row['FTR'] == 'H':
                away_wins += 1
            elif row['AwayTeam'] == away_team and row['FTR'] == 'A':
                away_wins += 1

        print(f"{home_team} wins in last {self.n} matches: {home_wins}")
        print(f"{away_team} wins in last {self.n} matches: {away_wins}")

    def print_team_lose_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_lose = 0
        for _, row in past_home.iterrows():
            if row['HomeTeam'] == home_team and row['FTR'] == 'A':
                home_lose += 1
            elif row['AwayTeam'] == home_team and row['FTR'] == 'H':
                home_lose += 1

        away_lose = 0
        for _, row in past_away.iterrows():
            if row['HomeTeam'] == away_team and row['FTR'] == 'A':
                away_lose += 1
            elif row['AwayTeam'] == away_team and row['FTR'] == 'H':
                away_lose += 1

        print(f"{home_team} losses in last {self.n} matches: {home_lose}")
        print(f"{away_team} losses in last {self.n} matches: {away_lose}")

    def print_team_draw_last_n(self):
        match = self.df.iloc[self.match_index - 1]
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        past_matches = self.df.iloc[:self.match_index - 1].copy()

        past_home = past_matches[
            (past_matches['HomeTeam'] == home_team) | (past_matches['AwayTeam'] == home_team)
        ].tail(self.n)

        past_away = past_matches[
            (past_matches['HomeTeam'] == away_team) | (past_matches['AwayTeam'] == away_team)
        ].tail(self.n)

        home_draws = sum(row['FTR'] == 'D' for _, row in past_home.iterrows())
        away_draws = sum(row['FTR'] == 'D' for _, row in past_away.iterrows())

        print(f"{home_team} draws in last {self.n} matches: {home_draws}")
        print(f"{away_team} draws in last {self.n} matches: {away_draws}")


In [ ]:
# Assuming `df` is your DataFrame and match index is 10
arpit = Arpit(df, match_index=381, n=5)
print('-----------------------------------------')
arpit = Arpit(df, match_index=381, n=15)
print('------------------------------------------')
arpit = Arpit(df, match_index=381, n=38)

Match 381: Arsenal vs Aston Villa
Arsenal goals in last 5 matches: 14
Aston Villa goals in last 5 matches: 8
Arsenal shots in last 5 matches: 62
Aston Villa shots in last 5 matches: 51
Arsenal shots on target in last 5 matches: 39
Aston Villa shots on target in last 5 matches: 28
Arsenal fouls in last 5 matches: 45
Aston Villa fouls in last 5 matches: 69
Arsenal corners in last 5 matches: 24
Aston Villa corners in last 5 matches: 28
Arsenal yellow cards in last 5 matches: 4
Aston Villa yellow cards in last 5 matches: 11
Arsenal red cards in last 5 matches: 0
Aston Villa red cards in last 5 matches: 0
Arsenal wins in last 5 matches: 4
Aston Villa wins in last 5 matches: 2
Arsenal losses in last 5 matches: 0
Aston Villa losses in last 5 matches: 3
Arsenal draws in last 5 matches: 1
Aston Villa draws in last 5 matches: 0
-----------------------------------------
Match 381: Arsenal vs Aston Villa
Arsenal goals in last 15 matches: 32
Aston Villa goals in last 15 matches: 16
Arsenal shots in

In [ ]:
import pandas as pd
from collections import defaultdict

# Load the original manipulated CSV (with all columns)
df

# Convert 'Date' column to datetime format for sorting
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# Sort by date to process chronologically
df.sort_values('Date', inplace=True)

# Create a backup of original column order to preserve it later
original_columns = df.columns.tolist()

# List of stats we want to track
stat_cols = ['FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

# Result conversion for easy count
def result_to_outcome(ftr, team, home_team):
    if ftr == 'H':
        return 'WIN' if team == home_team else 'LOSE'
    elif ftr == 'A':
        return 'WIN' if team != home_team else 'LOSE'
    elif ftr == 'D':
        return 'DRAW'
    return None

# Prepare team stats dictionary
team_stats = defaultdict(list)

# Set of window sizes
window_sizes = [5, 15, 38]

# Start filling stats
for idx, row in df.iterrows():
    home = row['HomeTeam']
    away = row['AwayTeam']
    ftr = row['FTR']

    for team, is_home in [(home, True), (away, False)]:
        past_matches = team_stats[team]

        for window in window_sizes:
            recent = past_matches[-window:] if len(past_matches) >= window else []

            # Prefixes
            loc = 'home' if is_home else 'away'

            # Initialize aggregation
            agg = defaultdict(int)

            # Count from recent matches
            for match in recent:
                for stat in stat_cols:
                    if stat in match:
                        agg[stat] += match[stat]

                outcome = match['Outcome']
                if outcome == 'WIN':
                    agg['WINS'] += 1
                elif outcome == 'LOSE':
                    agg['LOSES'] += 1
                elif outcome == 'DRAW':
                    agg['DRAWS'] += 1

            # Assign stats to correct columns in df
            for stat in stat_cols:
                df.at[idx, f"{stat}_L{window}_{loc}"] = agg[stat]

            # Outcome stats
            df.at[idx, f"WINS_L{window}_{loc}"] = agg['WINS']
            df.at[idx, f"LOSES_L{window}_{loc}"] = agg['LOSES']
            df.at[idx, f"DRAWS_L{window}_{loc}"] = agg['DRAWS']

    # Combined match row for historical record
    match_record_home = {
        'FTHG': row['FTHG'], 'FTAG': row['FTAG'], 'HS': row['HS'], 'HST': row['HST'],
        'HF': row['HF'], 'HC': row['HC'], 'HY': row['HY'], 'HR': row['HR'],
        'Outcome': result_to_outcome(ftr, home, home)
    }
    match_record_away = {
        'FTHG': row['FTAG'], 'FTAG': row['FTHG'], 'HS': row['AS'], 'HST': row['AST'],
        'HF': row['AF'], 'HC': row['AC'], 'HY': row['AY'], 'HR': row['AR'],
        'Outcome': result_to_outcome(ftr, away, home)
    }

    team_stats[home].append(match_record_home)
    team_stats[away].append(match_record_away)

    # Print progress every 100 rows
    if idx % 100 == 0:
        print(f"Processed row {idx+1}/{len(df)}")

# Fill any missing values (like first few matches) with 0
df.fillna(0, inplace=True)

new_columns = [col for col in df.columns if col not in original_columns]

# Reorder DataFrame
df = df[original_columns + new_columns]

# Save updated file
df.to_csv('/content/final_raw data.csv', index=False)
print("CSV file saved as 'final_updated_data.csv'")


<ipython-input-2-9beab5a9d656>:8: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
<ipython-input-2-9beab5a9d656>:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[idx, f"{stat}_L{window}_{loc}"] = agg[stat]
<ipython-input-2-9beab5a9d656>:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[idx, f"{stat}_L{window}_{loc}"] = agg[stat]
<ipython-input-2-

Processed row 101/3000
Processed row 201/3000
Processed row 301/3000
Processed row 401/3000
Processed row 501/3000
Processed row 601/3000
Processed row 701/3000
Processed row 801/3000
Processed row 901/3000
Processed row 1001/3000
Processed row 1101/3000
Processed row 1201/3000
Processed row 1301/3000
Processed row 1401/3000
Processed row 1501/3000
Processed row 1601/3000
Processed row 1701/3000
Processed row 1801/3000
Processed row 1901/3000
Processed row 2001/3000
Processed row 2101/3000
Processed row 2201/3000
Processed row 2301/3000
Processed row 2401/3000
Processed row 2501/3000
Processed row 2601/3000
Processed row 2701/3000
Processed row 2801/3000
Processed row 2901/3000
Processed row 3001/3000


<ipython-input-2-9beab5a9d656>:96: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


CSV file saved as 'final_updated_data.csv'


In [ ]:
from google.colab import files
files.download('/content/final_raw data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>